<a href="https://colab.research.google.com/github/Ilvecho/Happy-Customers/blob/main/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install Scrapy

In [2]:
import numpy as np
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess

from google.colab import files,drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Create the spider to crawl the desired URLs

In [3]:
class HrSpider(scrapy.Spider):
  name = 'hr_spider'

  def start_requests(self):
    urls = ['https://www.getimpactly.com/post/hr-compliance-checklist']

    for url in urls:
      yield scrapy.Request(url=url, callback=self.parse)

  # We want to preserve the order of the paragraphs and list items
  def combine_content(self, paragraphs, titles, list_items, link_text):
    combined_content = []

    # Add paragraphs and list items in the correct order
    while paragraphs or titles or list_items or link_text:
      if paragraphs:
        combined_content.append(paragraphs.pop(0))
      elif titles:
        combined_content.append(titles.pop(0))
      elif list_items:
        combined_content.append(list_items.pop(0))
      elif link_text:
        combined_content.append(link_text.pop(0))

    return combined_content

  def parse(self, response):
    output_text = ''
    # Extract content from <p> tags
    paragraphs = response.css('p::text').extract()

    # Extract content from <h2> tags
    titles = response.css('h2::text').extract()

    # Extract content from <li> tags
    list_items = response.css('li::text').extract()

    # Extract the text of links in the body -> XPath //p//a/text()
    link_text = response.xpath('//p/a/text()').extract()

    # Combine paragraphs and list items in the correct order
    combined_content = self.combine_content(paragraphs, titles, list_items, link_text)

    # Do something with the extracted content (e.g., print it)
    for content in combined_content:
      print(content)
      output_text = output_text + '\n' + content

    # Save the text in a file
    with open(r'/content/gdrive/MyDrive/hr_content.txt', 'w') as text_file:
      text_file.write(output_text)
      text_file.close()


In [4]:
process = CrawlerProcess()  # Look what Scrapy settings are # settings={'FEEDS': {'item.txt': {'format': 'txt'}}}
process.crawl(HrSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
2024-01-21 11:05:17 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-01-21 11:05:17 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.1.58+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-01-21 11:05:17 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGE

An HR compliance checklist is important for the proper functioning of an organization. HR compliance is defined as the responsibility of the business to abide by and follow the working standards set out by the employment facility. An HR compliance checklist is a systematically organized checklist that provides a list of tasks that need to be completed. Human resource specialists use it to prepare for an HR audit.
Compliance is required in the Human Resources department to define policies and procedures for ensuring that your employment and work practices signify a thorough understanding of all applicable laws, rules, and regulations while simultaneously also being aware of the company's larger objectives.
HR compliance is considered to be mandatory for the following reasons:
HR compliance officers need to ensure that every individual working in the company follows the policies and guidelines. They usually work alongside a dedicated HR compliance team for handling these regulations and 

In [5]:
process.stop()

<DeferredList at 0x7c9fd04bbfd0 current result: []>